In [1]:
library(tidyverse)
# remotes::install_github("bradleyboehmke/harrypotter")
library(harrypotter)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# Lecture 14: Regular expressions

<div style="border: 1px double black; padding: 10px; margin: 10px">

**After today's lecture you will:**
* Understand basic regular expressions.
* Use regular expressions to extract data from text.
</div>

These notes correspond to Chapter 16 of your book.


## Exercise
What is the last sentence of ch1? Hint: get a substring using `str_sub` of the last 200 characters and then find the index of `.` using `str_locate` in that and get the substring again using the index position

In [12]:
ch1 = philosophers_stone[1]

#### Revisiting escape sequence `\`

We see that the print statement does not quite display the newline on the console

In [13]:
print('abc \n abc')

[1] "abc \n abc"


However using `cat` function gets us the new line

In [22]:
cat('abc \n abc')

abc 
 abc

We can also use writeLines to get around this issue

In [41]:
writeLines('abc \n abc')

abc 
 abc


#### raw string to the rescue

Sometimes adding the escape sequence to escape a series of special characters can become very confusing. Let us understand this example

In [30]:
tricky <- "double_quote <- \"\\\"\" # or '\"'
single_quote <- '\\\'' # or \"'\""

cat(tricky)

double_quote <- "\"" # or '"'
single_quote <- '\'' # or "'"

In [31]:
simple <- r"(double_quote <- "\"" # or '"'
single_quote <- '\'' # or "'")"
cat(simple)

double_quote <- "\"" # or '"'
single_quote <- '\'' # or "'"

## Regular expressions
Regular expressions (regex, regexps) are a programming language that allows you to describe patterns in strings. They have a steep learning curve but are very powerful for working with text data. In this class we will just focus on the basics of regexps. A good tool for learning regexps is [regex101](https://regex101.com/), which lets you interactively edit and debug your regular expressions.

> Some people, when confronted with a problem, think "I know, I'll use regular expressions." Now they have two problems. 
>
> — Jamie Zawinski (famous nerd)

In these slides we will use the command `str_view` to understand how regular expressions work. 

The most basic regular expression is a plain string. It will match if the other string contains it as a substring.

In [2]:
x = c("apple", "banana", "pear") %>% print
str_view(x, pattern = "an")

[1] "apple"  "banana" "pear"  


[2] │ b<an><an>a

Here `str_view` has matched our regexp (`"an"`) inside of the second string `banana` of the vector `x`.

In [32]:
fruit %>% print

 [1] "apple"             "apricot"           "avocado"          
 [4] "banana"            "bell pepper"       "bilberry"         
 [7] "blackberry"        "blackcurrant"      "blood orange"     
[10] "blueberry"         "boysenberry"       "breadfruit"       
[13] "canary melon"      "cantaloupe"        "cherimoya"        
[16] "cherry"            "chili pepper"      "clementine"       
[19] "cloudberry"        "coconut"           "cranberry"        
[22] "cucumber"          "currant"           "damson"           
[25] "date"              "dragonfruit"       "durian"           
[28] "eggplant"          "elderberry"        "feijoa"           
[31] "fig"               "goji berry"        "gooseberry"       
[34] "grape"             "grapefruit"        "guava"            
[37] "honeydew"          "huckleberry"       "jackfruit"        
[40] "jambul"            "jujube"            "kiwi fruit"       
[43] "kumquat"           "lemon"             "lime"             
[46] "loquat"            

In [3]:
str_view(fruit, 'berry')

 [6] │ bil<berry>
 [7] │ black<berry>
[10] │ blue<berry>
[11] │ boysen<berry>
[19] │ cloud<berry>
[21] │ cran<berry>
[29] │ elder<berry>
[32] │ goji <berry>
[33] │ goose<berry>
[38] │ huckle<berry>
[50] │ mul<berry>
[70] │ rasp<berry>
[73] │ salal <berry>
[76] │ straw<berry>

### Wildcards
Our first non-trivial regular expression will use a wildcard: `.`. Used inside of a regular expression, the period matches any single character:

In [ ]:
str_view("else every eele etcetera", "e..e") 

[1] │ <else> every <eele> <etce>tera

If we want to "extract" the first match we can use `str_extract()` instead:

In [ ]:
str_extract("else every eele etcetera", "e..e ") 

[1] "else "

What about all the matches? You can guess the function here

In [34]:
str_extract_all("else every eele etcetera", "e..e") 

[[1]]
[1] "else" "eele" "etce"

Now let's return to another example using harrypotter: finding all capitalized words.

### Exercise
What is the first string that matches the pattern `H<any three characters>y` in Philosopher's Stone?

In [ ]:
# H<3>y

### Character classes
Instead of matching anything using `.`, we often want to match a class of things: words, numbers, spaces, etc.
A "character class" is a special pattern that matches a collection of characters. There are four built-in character classes you should know:
- `\w`: matches any alphanumeric character and is equivalent to using [A-Za-z0-9_]
- `\s`: matches single space, tab, newline characters and is equivalent to using [\t\n\r\f\v]
- `\d`: matches any digit and is equivalent to using [0-9]
- `\b`: match a "word boundary" (more on this in a moment).



`\w` matches any word character:

In [38]:
str_view("this is a word", "\\w")

[1] │ <t><h><i><s> <i><s> <a> <w><o><r><d>

Note the additional level of escaping needed here: "\\w" gets parsed by R into the string `\w`:

In [ ]:
writeLines("\\w")

\w


The `\w` is then parsed again by the regular expression.

The string "\w" is not valid in R, because there is no escape code "\w":

```
> "\w"
Error: '\w' is an unrecognized escape in character string starting ""\w"
Traceback:
```

In [40]:
# raw representation
str_view("this is a word", r"(\w)")

[1] │ <t><h><i><s> <i><s> <a> <w><o><r><d>

`\d` will match any digit:

In [ ]:
str_view(c("number1", "two", "3hree"), "\\d")

[1] │ number<1>
[3] │ <3>hree

Similarly, `\s` will match whitespace (spaces, tabs and newlines):

In [ ]:
y = c("spa ce", "hello\tworld", "multi\nline")
writeLines(y)
str_view(y, "\\s")

spa ce
hello	world
multi
line


[1] │ spa< >ce
[2] │ hello<{\t}>world
[3] │ multi<
    │ >line

You can also create your own character class using square brackets: `[abc]` will match *one of* `a`, `b`, or `c`. In other words, the 'width' of a character class is one character by default.

In [ ]:
str_view(fruit, '[be]a')  # Match either 'b' or 'e' followed by a

 [4] │ <ba>nana
[12] │ br<ea>dfruit
[58] │ p<ea>ch
[59] │ p<ea>r
[62] │ pin<ea>pple

We can use character classes to match the first capital letter of a capitalized word:

In [ ]:
str_view(c("These", "are", "some Capitalized words"),
         "[ABCDEFGHIJKLMNOPQRSTUVWXYZ]")

[1] │ <T>hese
[3] │ some <C>apitalized words

We do not need to go to all the trouble of typing each capital letter. We can use the shortcut `[A-Z]` instead.

In [ ]:
str_view(c("These", "are", "some Capitalized words"), "[A-Z]")

[1] │ <T>hese
[3] │ some <C>apitalized words

### Word boundaries
A final character class we'll use frequently is `\b`, which stands for "word boundary". A word boundary matches the "edges" of a word:

In [ ]:
str_view(c("Rafael Nadal", "Roger Federer", "Novak Djokovic"), "\\b")

[1] │ <>Rafael<> <>Nadal<>
[2] │ <>Roger<> <>Federer<>
[3] │ <>Novak<> <>Djokovic<>

Every word has a word boundary on either side, so we can use this in combination with other character classes to match certain kinds of words in text.

## 🤔 Quiz

About how many words start with 'H' in `ch1`?

<ol style="list-style-type: upper-alpha;">
    <li>98</li>
    <li>100</li>
    <li>99</li>
    <li>1</li>
</ol>


In [42]:
#words starting with 'H'


In this exercise, we matched the pattern 

    <word boundary><character H><any character (.)>

## Quantifiers
Now we can return to a question that we asked in the previous lecture: how many words are there in `ch1`? We did a crude approximation by counting the number of spaces, but we saw that this was not quite accurate when we used `\\w+` to find words. Now let us understand this second expression in detail

            \\w+ - <any number of word characters being together>

The four quantifiers you should know are:
- `?`: match zero or one of the preceding character.
- `+`: match one or more of the preceding character.
- `*`: match zero or more of the preceding character.
- `{x}`: match exactly `x` of the preceding character.
    - `{x,y}`: match between `x` and `y` of the preceding character.
    - `{x,}`: match at least `x` of the preceding character.

So, to count the number of words using the pattern shown above:

In [44]:
# match any number of word characters 

## 🤔 Quiz

How many words in `ch1` match the pattern:

    <word boundary><small case 'h'><any exactly 5 characters><word boundary>

(Example: `harry.` matches but ` I ` does not.)

<ol style="list-style-type: upper-alpha;">
    <li>100</li>
    <li>19</li>
    <li>98</li>
    <li>51</li>
</ol>


In [ ]:
# 

Find all the character names in Harry Potter. By matching all words that start with capital letters, we're off to a good start. But we pick up the beginning word of any sentence, resulting in a lot of false matches. Let's use quantifiers to restrict to only longer words, say capitalized words that are at least six characters long.

Our pattern becomes:

    <word boundary><Capital letter><at least five lowercase letters><word boundary>

In [ ]:
# all cap words that have at least six characters

## Grouping
"Professor" is one of the most common capitalized words in this book. Is there a character named Professor, or is it just a title? Now let us try to match one or more capitalized words in a row. We can accomplish this by creating a *group*, and then applying a quantifier to it. 

To create a group, I surround a part of my regexp with parentheses:

In [ ]:
str_view("this will be grouped", "[a-z]+ ?")
str_view("this will be grouped", "([a-z]+ ?)")

The parentheses do not change the regular expression (but they are doing something else, which we will discuss shortly.) But now I can apply a quantifier to the whole group:

In [ ]:
str_view("this will be grouped", "([a-z]+ ?)+")

So now we take the previous pattern and group it:

    (<word boundary><Capital letter><at least five lowercase letters><word boundary>){match 1+ times}

In [45]:
# match one or more cap words in a row
str_extract_all(ch1, '([A-Z][a-z]{5,} ?)+') %>% table

.
           Although              Bonfire             Borrowed  
                   1                    1                    1 
             Bristol              Britain               Couldn 
                   1                    1                    1 
      Dedalus Diggle               Dudley              Dudley  
                   1                    2                    7 
          Dumbledore          Dumbledore               Dundee  
                  16                   19                    1 
             Dursley             Dursley           Dursleyish  
                   9                   38                    1 
            Dursleys            Dursleys             Everyone  
                   2                    5                    1 
             Exactly             Experts               Famous  
                   1                    1                    2 
             Flocks                Godric             Gonagall 
                   1                  

## Negations
Earlier we looked at quotations. The first quotation in chapter 1 is:

In [46]:
str_sub(ch1, 2150, 2163)

[1] "\"Little tyke,\""

How can we find other quotes? The pattern for a quote is a quotation mark, followed by any number of things that are not a quotation mark, followed by another quotation mark:

    <quotation mark><anything that is not a quotation mark><quotation mark>


To match this, we will use a *negation*. A negation is a character class that begins with the character "^". It matches anything that in *not* inside the character class:

In [ ]:
str_view_all("match doesn't match", "[^aeiou]+")

[1] │ <m>a<tch d>oe<sn't m>a<tch>

To match a quotation, we'll input the pattern that we specified above:

In [ ]:
str_view_all('"Here is a quotation", said the professor. "And here is another."',
             '"[^"]+"')

[1] │ <"Here is a quotation">, said the professor. <"And here is another.">

## Backreferences
Parentheses define groups that can be referred to later in the match as `\1`, `\2` etc. This is called a backreference. For example:

    (.)\1

will match the same character repeated twice in a row:

In [48]:
"eel"  %>% str_view("(.)\\1")

[1] │ <ee>l

## 🤔 Quiz

What does this regular expression match?:

```
(..).*\\1
```


<ol style="list-style-type: upper-alpha;">
    <li>Any word that starts and ends with the same character, e.g. `alpha`</li>
    <li>Any word that starts and ends with the same two characters, e.g. `church`</li>
    <li>Any word that ends with two characters that are found earlier in the string, e.g. `therefore`</li>
    <li>I hate regular expressions.</li>
</ol>

In [ ]:
# what's it match?

## Anchors
Sometimes we want a match to occur at a particular position in the string. For example, "all words which start with b". For this we have the special anchor characters: `^` and `$`. The caret `^` matches the beginning of a string. The `$` matches the end.

In [ ]:
x <- c('apple', 'banana', 'pear')
str_view(x, '^b')
str_view(x, 'r$')

HTML widgets cannot be represented in plain text (need html)

HTML widgets cannot be represented in plain text (need html)

## 🤔 Quiz

What does this regular expression match?:

```
^(..).*\\1$
```


<ol style="list-style-type: upper-alpha;">
    <li>Any word that starts and ends with the same character, e.g. `alpha`</li>
    <li>Any word that starts and ends with the same two characters, e.g. `church`</li>
    <li>Any word that ends with a character that is found earlier in the string, e.g. `therefore`</li></li>
    <li>I hate regular expressions.</li>
</ol>

In [ ]:
# solution

### `str_match`
`str_match(v, re)` will create a matrix out of the grouped matches in `re`. 
The first column has the whole match, and additional columns are added for each character group. 
If the pattern does not match, you will get `NA`s.

In [ ]:
head(str_match(words, '^(.).*(.)$'))

NA,NA,NA
able,a,e
about,a,t
absolute,a,e
accept,a,t
account,a,t


### `str_replace`
`str_replace(v, re, rep)` will replace each match of `re` in `v` with `rep`. The most basic usage is as a sort of find and replace:

In [ ]:
str_replace('Give me liberty or give me death', '\\w+$', 'pizza')

[1] "Give me liberty or give me pizza"